In [7]:
import pandas as pd
import ast
import re
import pickle as pk

In [8]:
e_map = {'PER' : '사람', 'ORG' : '단체', 'LOC' : '지명', 'POH' : '나머지', 'DAT' : '날짜', 'NOH' : '수량'}

In [9]:
def convert(sen,sub,obj,e_map):
    sub = ast.literal_eval(sub)
    obj = ast.literal_eval(obj)
    e1_str = '<e1>'
    e1_end = '</e1>'
    e2_str = '<e2>'
    e2_end = '</e2>'

    str_list = list(sen)

    str_list[sub['start_idx']] = e1_str + ' <e3> ' +e_map[sub['type']] + ' <e3> ' +str_list[sub['start_idx']]
    str_list[sub['end_idx']] = str_list[sub['end_idx']] + ' ' + e1_end + ' '

    str_list[obj['start_idx']] = e2_str + ' <e3> ' +e_map[obj['type']] + ' <e3> ' + str_list[obj['start_idx']]
    str_list[obj['end_idx']] = str_list[obj['end_idx']] + ' ' + e2_end + ' '
    
    pre_str = ''.join(str_list)

    return pre_str

In [10]:
TRAIN_PATH = '/opt/ml/dataset/train/cleaned_target_augmented.csv'
TEST_PATH = '/opt/ml/dataset/test/test_data.csv'
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

In [11]:
pre_train_df = train_df.apply(lambda x : convert(x.sentence, x.subject_entity, x.object_entity,e_map), axis= 1)
pre_test_df = test_df.apply(lambda x : convert(x.sentence, x.subject_entity, x.object_entity,e_map), axis= 1)

In [12]:
# pre_train_df = train_df['sentence']
# pre_test_df = test_df['sentence']

In [13]:
def del_spe_change_token(s):
    rs = re.sub(r'[^a-zA-Z0-9가-힇ㄱ-ㅎㅏ-ㅣぁ-ゔァ-ヴー々〆〤一-龥,".?!<>/ ]+', ' ' ,s)
    
    rs = rs.replace('<e1>','#')
    rs = rs.replace('</e1>','#' )
    rs = rs.replace('<e2>','@' )
    rs = rs.replace('</e2>','@' )
    rs = rs.replace('<e3>',' ' )
    

    rs = rs.replace('<','')
    rs = rs.replace('>','')

    return rs

In [14]:
pre_train_df = pre_train_df.apply(del_spe_change_token)
pre_test_df = pre_test_df.apply(del_spe_change_token)

In [15]:
train_df['sentence'] = pre_train_df
test_df['sentence'] = pre_test_df

In [16]:
train_df = train_df[['id','sentence','subject_entity','object_entity','label','source']]


In [17]:
train_df.head()

,id,sentence,subject_entity,object_entity,label,source
0,0,Something 는 @ * 사람 * 조지 해리슨 @ 이 쓰고 # * 단체 * 비...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당 @ * 단체 * 대안신당 @ # * 단체 * 민주평화당 ...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 # * 단체 * 광주FC # 는 지난 26일 ...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 주 # * 단체 * 아성다이소 # 대표 @ * 사람 * 박정부...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,@ * 날짜 * 1967 @ 년 프로 야구 드래프트 1순위로 # * 단체 * 요미우...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia


In [22]:
train_df.to_csv('/opt/ml/dataset/train/preprocess_train.csv',index=False)
test_df.to_csv('/opt/ml/dataset/test/preprocess_test.csv',index=False)

In [12]:
def make_label1(s):
    split_s = s.split(':')
    return split_s[0]

def make_label2(s):
    split_s = s.split(':')
    if len(split_s) == 1:
        return
    else:

        return split_s[1]


In [13]:
train_df['label1'] = train_df['label'].apply(make_label1)

In [14]:
train_df['label2'] = train_df['label'].apply(make_label2)

In [15]:
org_train = train_df[train_df['label1'] == 'org']
per_train = train_df[train_df['label1'] == 'per']

In [39]:
org_list = list(org_train['label2'].unique())

org_label_to_num = {}
org_num_to_label = {}
for i, j in enumerate(org_list):
    org_label_to_num[j] = i
    org_num_to_label[i] = j

In [40]:
per_list = list(per_train['label2'].unique())

per_label_to_num = {}
per_num_to_label = {}
for i, j in enumerate(per_list):
    per_label_to_num[j] = i
    per_num_to_label[i] = j

In [41]:
train_list = list(train_df['label1'].unique())

rel_label_to_num = {}
rel_num_to_label = {}
for i, j in enumerate(train_list):
    rel_label_to_num[j] = i
    rel_num_to_label[i] = j

In [42]:
with open("org_label_to_num.pkl","wb") as fw:
    pk.dump(org_label_to_num, fw)

with open("org_num_to_label.pkl","wb") as fw:
    pk.dump(org_num_to_label, fw)

with open("per_label_to_num.pkl","wb") as fw:
    pk.dump(per_label_to_num, fw)

with open("per_num_to_label.pkl","wb") as fw:
    pk.dump(per_num_to_label, fw)

with open("rel_label_to_num.pkl","wb") as fw:
    pk.dump(rel_label_to_num, fw)

with open("rel_num_to_label.pkl","wb") as fw:
    pk.dump(rel_num_to_label, fw)

In [43]:
org_train['label'] = org_train['label2']
per_train['label'] = per_train['label2']

<ipython-input-43-8936fb9b075b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  org_train['label'] = org_train['label2']
<ipython-input-43-8936fb9b075b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_train['label'] = per_train['label2']


In [44]:
train_df['label'] = train_df['label1']

In [45]:
org_train = org_train[['id','sentence','subject_entity','object_entity','label','source']]
per_train = per_train[['id','sentence','subject_entity','object_entity','label','source']]

In [46]:
test_df.to_csv('/opt/ml/dataset/test/preprocess_test.csv',index=False)

In [47]:
org_train.to_csv('/opt/ml/dataset/train/org_train.csv',index=False)
per_train.to_csv('/opt/ml/dataset/train/per_train.csv',index=False)
train_df.to_csv('/opt/ml/dataset/train/rel_train.csv',index=False)

In [48]:
train_df['label'].value_counts()

per            12654
org            10282
no_relation     9534
Name: label, dtype: int64

In [49]:
len(org_train['label'].unique())
print(len(per_train['label'].unique()))

18


In [50]:
per_train['label'].value_counts()

employee_of           3573
title                 2103
origin                1234
date_of_birth         1130
alternate_names       1001
spouse                 795
colleagues             534
parents                520
date_of_death          418
children               304
place_of_residence     193
other_family           190
place_of_birth         166
product                139
siblings               136
religion                96
schools_attended        82
place_of_death          40
Name: label, dtype: int64

In [51]:
rel_label_to_num

{'no_relation': 0, 'org': 1, 'per': 2}

In [52]:
rel_label_to_num

{'no_relation': 0, 'org': 1, 'per': 2}

In [53]:
per_label_to_num

{'date_of_birth': 0,
 'employee_of': 1,
 'origin': 2,
 'title': 3,
 'schools_attended': 4,
 'colleagues': 5,
 'alternate_names': 6,
 'spouse': 7,
 'children': 8,
 'place_of_birth': 9,
 'parents': 10,
 'religion': 11,
 'date_of_death': 12,
 'place_of_residence': 13,
 'other_family': 14,
 'siblings': 15,
 'product': 16,
 'place_of_death': 17}

In [54]:
org_label_to_num

{'member_of': 0,
 'top_members/employees': 1,
 'alternate_names': 2,
 'place_of_headquarters': 3,
 'members': 4,
 'founded_by': 5,
 'political/religious_affiliation': 6,
 'founded': 7,
 'number_of_employees/members': 8,
 'dissolved': 9,
 'product': 10}

In [55]:
org_train['sentence']

2        K리그2에서 성적 1위를 달리고 있는 #  * 단체 * 광주FC # 는 지난 26일...
3        균일가 생활용품점  주 #  * 단체 * 아성다이소 #  대표 @ * 사람 * 박정...
5          유엔, 유럽 의회, #  * 단체 * 북대서양 조약 기구 #   @ * 단체 *...
8        중공군에게 온전  대항할 수 없을 정도로 약해진 국민당은 @ * 지명 * 타이베이 ...
12       2009년 9월, 미국 프로 야구 #  * 단체 * 필라델피아 필리스 #  소속의 ...
                               ...                        
32453    1952년 8월에 실시된 제2대 대통령 선거 후 집권당인 #  * 단체 * 자유당 ...
32455    김홍균 연구원은  일회성 손익으로 영업단에 신규 수주호선 및 공사손실충당금 118억...
32458    대학 졸업 직후, 그는  #  * 단체 * 스티어 프라이데이 나이트 #  @ * 나...
32467    #  * 단체 * 완도군 #  군수 @ * 사람 * 신우철 @  이 국토교통부에서 ...
32469    #  * 단체 * 화순군 #  군수 @ * 사람 * 구충곤 @  은 17일 동면의 ...
Name: sentence, Length: 10282, dtype: object

In [56]:
test_df

,id,sentence,subject_entity,object_entity,label,source
0,0,지난 15일 # * 단체 * MBC # @ * 단체 * 탐사기획 스트레이트 @...,"{'word': 'MBC', 'start_idx': 7, 'end_idx': 9, ...","{'word': '탐사기획 스트레이트', 'start_idx': 12, 'end_i...",100,wikitree
1,1,사랑스러운 # * 사람 * 프린세스 프링 # 의 이름은 봄의 공주님 Prin...,"{'word': '프린세스 프링', 'start_idx': 7, 'end_idx':...","{'word': '공주', 'start_idx': 84, 'end_idx': 85,...",100,wikipedia
2,2,"한편, 본인 이근안 을 모델로 한 MBC 특집드라마 가 @ * 날짜 * 1995년 ...","{'word': '경찰', 'start_idx': 121, 'end_idx': 12...","{'word': '1995년', 'start_idx': 31, 'end_idx': ...",100,wikipedia
3,3,@ * 사람 * 정창손 @ 은 김질과 같이 대궐로 달려가 고변하며 신은 실로 모르...,"{'word': '세조', 'start_idx': 78, 'end_idx': 79,...","{'word': '정창손', 'start_idx': 0, 'end_idx': 2, ...",100,wikipedia
4,4,당시 # * 단체 * 민주당 # 이진련 시의원은 @ * 사람 * 권영진 @ 시...,"{'word': '민주당', 'start_idx': 3, 'end_idx': 5, ...","{'word': '권영진', 'start_idx': 16, 'end_idx': 18...",100,wikitree
...,...,...,...,...,...,...
7760,7760,코로나19 방역 조치의 일환으로 국민의 움직임을 통제하려는 # * 단체 * 정부 ...,"{'word': '정부', 'start_idx': 33, 'end_idx': 34,...","{'word': '이탈리아', 'start_idx': 41, 'end_idx': 4...",100,wikitree
7761,7761,선 연구원은 위식도역류질환치료제인 케이캡이 92억원 판매되면서 2019년 연간 3...,"{'word': '종근당', 'start_idx': 133, 'end_idx': 1...","{'word': '전년', 'start_idx': 143, 'end_idx': 14...",100,wikitree
7762,7762,# * 단체 * 한국전기안전공사 # 사장 @ * 사람 * 조성완 @ 는 8월 ...,"{'word': '한국전기안전공사', 'start_idx': 0, 'end_idx'...","{'word': '조성완', 'start_idx': 12, 'end_idx': 14...",100,wikitree
7763,7763,@ * 날짜 * 1987년 @ # * 사람 * B. 슈나이더 # B. Schn...,"{'word': 'B. 슈나이더', 'start_idx': 6, 'end_idx':...","{'word': '1987년', 'start_idx': 0, 'end_idx': 4...",100,wikipedia
